# **LLM 자세히 살펴보기**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



In [ ]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

## **LLM 로드하기**

- **pipeline** : 복잡한 전처리/후처리 과정을 자동화해주는 고수준 API

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 모델과 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

# Create a pipeline
generator = pipeline(
    "text-generation",      # 수행할 작업(task) 유형
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # 생성된 텍스트만 반환할지, 입력 프롬프트를 포함할지 결정(프롬프트 제외)
    max_new_tokens=50,      # 생성할 새로운 토큰의 최대 개수
    do_sample=False,        # 샘플링(무작위성) 사용 여부
)



---



## **훈련된 트랜스포머 LLM의 입력과 출력**


In [ ]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)

print(output[0]['generated_text'])


### **[실습] max_new_tokens 값 조정하기**
max_new_tokens 값을 크게 늘려서 결과 확인하기
- max_new_tokens = 200
- max_new_tokens = 1000 (--> OutOfMemoryError)

In [ ]:
print(model)



---



## **확률 분포로부터 하나의 토큰 선택하기(샘플링/디코딩)**

- 확률 분포에서 하나의 토큰을 선택하는 방법 --> **디코딩 전략**
- 확률 점수가 가장 높은 토큰을 고르는 경우 --> **Greedy Decoding(탐욕적 디코딩)**
    - **LLM의 온도(temperature) 매개변수를 0으로 설정하는 방식**

In [ ]:
prompt = "The capital of France is"

# 입력 프롬프트를 토큰화합니다.
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# 입력 토큰을 GPU에 배치합니다.
input_ids = input_ids.to("cuda")

# lm_head 앞에 있는 model의 출력을 얻습니다.
model_output = model.model(input_ids)

# lm_head의 출력을 얻습니다.
lm_head_output = model.lm_head(model_output[0])

In [ ]:
lm_head_output

In [ ]:
# 첫 번째 데이터(문장)의 마지막 토큰(단어) 추출하기
token_id = lm_head_output[0,-1].argmax(-1) # 마지막에 생성된 토큰ID
tokenizer.decode(token_id)

In [ ]:
model_output[0].shape

In [ ]:
lm_head_output.shape

## **키와 값을 캐싱(kv cache)하여 생성 속도 높이기**


In [ ]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

# 입력 프롬프트를 토큰화합니다.
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

- 문장 생성 속도 측정(캐싱 사용)

In [ ]:
%%timeit -n 1
# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=True
)

- 문장 생성 속도 측정(캐싱 사용 안함)

In [ ]:
%%timeit -n 1
# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=False
)



---



## **멀티헤드, 그룹쿼리, 멀티쿼리**
- 프롬프트 : 허깅페이스에 있는 모델 중 멀티헤드, 그룹쿼리, 멀티쿼리를 사용한 모델명을 각각 2개씩 알려줘.

### **멀티헤드 어텐션 (Multi-Head Attention, MHA)**
- 입력을 여러 개의 독립적인 "헤드"로 나누어 각각 다른 표현 부분공간(representation subspace)을 학습하게 함.
- 각 헤드는 Query, Key, Value를 독립적으로 가지며, 서로 다른 패턴을 포착할 수 있음

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch

class MHAModelExplorer:
    """
    Multi-Head Attention 모델 탐색 및 분석
    """
    def __init__(self):
        pass

    def load_bert_mha(self):
        """BERT의 MHA 구조 분석"""
        print("\n" + "=" * 60)
        print("🔍 BERT (Multi-Head Attention) 분석")
        print("=" * 60)

        # 모델 로드
        model_name = 'bert-base-uncased'
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)

        # 모델 구조 분석
        print(f"\n📊 모델: {model_name}")
        print(f"총 파라미터: {sum(p.numel() for p in model.parameters()):,}")

        # Attention 설정 확인
        config = model.config
        print(f"\n🎯 Attention 설정:")
        print(f"   Attention 헤드 수: {config.num_attention_heads}")
        print(f"   Hidden size: {config.hidden_size}")
        print(f"   헤드당 차원: {config.hidden_size // config.num_attention_heads}")

        # 각 레이어의 Attention 구조
        print(f"\n🏗️  아키텍처:")
        print(f"   Encoder 레이어 수: {config.num_hidden_layers}")
        print(f"   각 레이어마다 {config.num_attention_heads}개의 독립적인 Attention 헤드")

        # Multi-Head Attention 특징
        print(f"\n✨ Multi-Head Attention 특징:")
        print(f"   ✅ Query 헤드: {config.num_attention_heads}개 (독립)")
        print(f"   ✅ Key 헤드: {config.num_attention_heads}개 (독립)")
        print(f"   ✅ Value 헤드: {config.num_attention_heads}개 (독립)")
        print(f"   → 총 KV 캐시: {config.num_attention_heads} 세트")

        # 실제 사용 예시
        text = "Transformers revolutionized natural language processing."
        inputs = tokenizer(text, return_tensors='pt')

        with torch.no_grad():
            outputs = model(**inputs, output_attentions=True)

        # Attention weights 분석
        attentions = outputs.attentions  # 각 레이어의 attention weights
        print(f"\n📐 Attention Weights 형태:")
        print(f"   레이어 수: {len(attentions)}")
        print(f"   각 레이어 attention shape: {attentions[0].shape}")
        print(f"   → [batch_size, num_heads, seq_len, seq_len]")

        print("=" * 60)

        return model, tokenizer

    def load_gpt2_mha(self):
        """GPT-2의 MHA 구조 분석"""
        print("\n" + "=" * 60)
        print("🔍 GPT-2 (Multi-Head Attention) 분석")
        print("=" * 60)

        # 모델 로드
        model_name = 'gpt2'
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        # 모델 구조 분석
        print(f"\n📊 모델: {model_name}")
        print(f"총 파라미터: {sum(p.numel() for p in model.parameters()):,}")

        # Attention 설정 확인
        config = model.config
        print(f"\n🎯 Attention 설정:")
        print(f"   Attention 헤드 수: {config.n_head}")
        print(f"   Hidden size: {config.n_embd}")
        print(f"   헤드당 차원: {config.n_embd // config.n_head}")

        print(f"\n🏗️  아키텍처:")
        print(f"   Decoder 레이어 수: {config.n_layer}")
        print(f"   각 레이어마다 {config.n_head}개의 독립적인 Attention 헤드")

        print(f"\n✨ Multi-Head Attention 특징:")
        print(f"   ✅ Query 헤드: {config.n_head}개 (독립)")
        print(f"   ✅ Key 헤드: {config.n_head}개 (독립)")
        print(f"   ✅ Value 헤드: {config.n_head}개 (독립)")
        print(f"   → Causal Masking 적용 (자기회귀적 생성)")

        # 메모리 사용량 추정
        head_dim = config.n_embd // config.n_head
        print(f"\n💾 KV 캐시 메모리 (시퀀스 길이 1024 기준):")
        kv_cache_size = 2 * config.n_layer * config.n_head * 1024 * head_dim * 2  # 2 bytes (FP16)
        print(f"   → 약 {kv_cache_size / 1e6:.2f} MB")

        print("=" * 60)

        return model, tokenizer

    def compare_mha_models(self):
        """MHA 모델들 비교"""
        print("\n" + "=" * 60)
        print("📊 Multi-Head Attention 모델 비교")
        print("=" * 60)

        models_info = [
            {
                'name': 'BERT-base',
                'hf_name': 'bert-base-uncased',
                'heads': 12,
                'hidden': 768,
                'layers': 12,
                'params': '110M',
                'type': 'Encoder-only'
            },
            {
                'name': 'GPT-2',
                'hf_name': 'gpt2',
                'heads': 12,
                'hidden': 768,
                'layers': 12,
                'params': '124M',
                'type': 'Decoder-only'
            },
            {
                'name': 'RoBERTa-base',
                'hf_name': 'roberta-base',
                'heads': 12,
                'hidden': 768,
                'layers': 12,
                'params': '125M',
                'type': 'Encoder-only'
            },
            {
                'name': 'T5-base',
                'hf_name': 't5-base',
                'heads': 12,
                'hidden': 768,
                'layers': 12,
                'params': '220M',
                'type': 'Encoder-Decoder'
            }
        ]

        print("\n추가 MHA 모델 목록:")
        print("-" * 60)
        for model in models_info:
            print(f"\n🤖 {model['name']}")
            print(f"   HF 모델명: {model['hf_name']}")
            print(f"   헤드 수: {model['heads']}, Hidden: {model['hidden']}")
            print(f"   레이어: {model['layers']}, 파라미터: {model['params']}")
            print(f"   타입: {model['type']}")

        print("\n" + "=" * 60)
        print("💡 MHA의 장단점")
        print("=" * 60)
        print("✅ 장점:")
        print("   - 각 헤드가 독립적으로 다른 패턴 학습")
        print("   - 풍부한 표현력")
        print("   - 병렬 처리 가능")
        print("\n⚠️  단점:")
        print("   - KV 캐시 메모리 사용량 높음")
        print("   - 긴 시퀀스에서 추론 속도 느림")
        print("   - 대형 모델에서 메모리 병목")
        print("=" * 60)

# ========================================
# 사용 예시
# ========================================

explorer = MHAModelExplorer()

# BERT 분석
bert_model, bert_tokenizer = explorer.load_bert_mha()

# GPT-2 분석
gpt2_model, gpt2_tokenizer = explorer.load_gpt2_mha()

# 모델 비교
explorer.compare_mha_models()

### **Grouped-Query Attention (GQA) 모델**
- Multi-Head Attention의 표현력과 Multi-Query Attention의 효율성을 결합한 절충안
- Query 헤드를 여러 그룹으로 나누고, 각 그룹이 하나의 Key-Value 헤드를 공유함
    - 2023년 LLM이 점점 커지면서 추론 시 KV 캐시가 주요 병목이 됨
    - MQA는 빠르지만 품질 저하 우려
    - GQA는 8개 정도의 KV 헤드로도 MHA와 유사한 성능 달성

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

class GQAModelExplorer:
    """
    Grouped-Query Attention 모델 탐색 및 분석
    """
    def __init__(self):
        pass

    def load_llama2_gqa(self):
        """Llama 2의 GQA 구조 분석"""
        print("\n" + "=" * 60)
        print("🔍 Llama 2 70B (Grouped-Query Attention) 분석")
        print("=" * 60)

        # 참고: 실제 70B 모델은 메모리가 많이 필요하므로 설정만 확인
        model_name = 'meta-llama/Llama-2-70b-hf'

        print(f"\n📊 모델: {model_name}")
        print(f"총 파라미터: 70B")

        # GQA 설정 (Llama 2 70B 기준)
        num_attention_heads = 64
        num_key_value_heads = 8  # GQA의 핵심!
        hidden_size = 8192
        num_layers = 80

        print(f"\n🎯 GQA Attention 설정:")
        print(f"   Query 헤드 수: {num_attention_heads}")
        print(f"   KV 헤드 수 (그룹): {num_key_value_heads}")
        print(f"   그룹 수: {num_key_value_heads}")
        print(f"   그룹당 Query 헤드: {num_attention_heads // num_key_value_heads}")
        print(f"   Hidden size: {hidden_size}")

        print(f"\n🏗️  아키텍처:")
        print(f"   Decoder 레이어 수: {num_layers}")

        print(f"\n✨ Grouped-Query Attention 특징:")
        print(f"   ✅ Query 헤드: {num_attention_heads}개 (독립)")
        print(f"   ✅ Key 헤드: {num_key_value_heads}개 (그룹으로 공유)")
        print(f"   ✅ Value 헤드: {num_key_value_heads}개 (그룹으로 공유)")
        print(f"   → 각 KV 헤드를 {num_attention_heads // num_key_value_heads}개의 Query 헤드가 공유")

        # 메모리 효율성 계산
        head_dim = hidden_size // num_attention_heads
        seq_len = 4096  # Llama 2의 컨텍스트 길이

        # MHA vs GQA 메모리 비교
        mha_kv_cache = 2 * num_layers * num_attention_heads * seq_len * head_dim * 2  # FP16
        gqa_kv_cache = 2 * num_layers * num_key_value_heads * seq_len * head_dim * 2  # FP16

        print(f"\n💾 KV 캐시 메모리 비교 (시퀀스 길이 {seq_len}):")
        print(f"   MHA 방식: {mha_kv_cache / 1e9:.2f} GB")
        print(f"   GQA 방식: {gqa_kv_cache / 1e9:.2f} GB")
        print(f"   절감량: {(1 - gqa_kv_cache / mha_kv_cache) * 100:.1f}%")
        print(f"   → {num_attention_heads // num_key_value_heads}배 메모리 효율!")

        print("=" * 60)

    def load_mistral_gqa(self):
        """Mistral 7B의 GQA 구조 분석"""
        print("\n" + "=" * 60)
        print("🔍 Mistral 7B (Grouped-Query Attention) 분석")
        print("=" * 60)

        model_name = 'mistralai/Mistral-7B-v0.1'

        print(f"\n📊 모델: {model_name}")

        # Mistral 7B의 GQA 설정
        num_attention_heads = 32
        num_key_value_heads = 8  # GQA
        hidden_size = 4096
        num_layers = 32

        print(f"총 파라미터: 7.24B")

        print(f"\n🎯 GQA Attention 설정:")
        print(f"   Query 헤드 수: {num_attention_heads}")
        print(f"   KV 헤드 수 (그룹): {num_key_value_heads}")
        print(f"   그룹 수: {num_key_value_heads}")
        print(f"   그룹당 Query 헤드: {num_attention_heads // num_key_value_heads}")
        print(f"   Hidden size: {hidden_size}")

        print(f"\n🏗️  아키텍처:")
        print(f"   Decoder 레이어 수: {num_layers}")
        print(f"   Sliding Window Attention: 4096 토큰")

        print(f"\n✨ Grouped-Query Attention 특징:")
        print(f"   ✅ Query 헤드: {num_attention_heads}개")
        print(f"   ✅ KV 헤드: {num_key_value_heads}개")
        print(f"   ✅ 그룹화 비율: {num_attention_heads // num_key_value_heads}:1")

        # 메모리 효율성
        head_dim = hidden_size // num_attention_heads
        seq_len = 8192  # Mistral의 슬라이딩 윈도우

        mha_kv_cache = 2 * num_layers * num_attention_heads * seq_len * head_dim * 2
        gqa_kv_cache = 2 * num_layers * num_key_value_heads * seq_len * head_dim * 2

        print(f"\n💾 KV 캐시 메모리 비교:")
        print(f"   MHA 방식: {mha_kv_cache / 1e9:.2f} GB")
        print(f"   GQA 방식: {gqa_kv_cache / 1e9:.2f} GB")
        print(f"   절감량: {(1 - gqa_kv_cache / mha_kv_cache) * 100:.1f}%")

        print(f"\n🚀 추가 최적화:")
        print(f"   ✅ Sliding Window Attention")
        print(f"   ✅ RoPE (Rotary Position Embedding)")
        print(f"   ✅ SwiGLU Activation")

        print("=" * 60)

    def visualize_gqa_concept(self):
        """GQA 개념 시각화"""
        print("\n" + "=" * 60)
        print("📊 GQA vs MHA vs MQA 개념 비교")
        print("=" * 60)

        print("\n1️⃣ Multi-Head Attention (MHA)")
        print("   Query: [H1] [H2] [H3] [H4] [H5] [H6] [H7] [H8]")
        print("   Key:   [H1] [H2] [H3] [H4] [H5] [H6] [H7] [H8]")
        print("   Value: [H1] [H2] [H3] [H4] [H5] [H6] [H7] [H8]")
        print("   → 8개의 독립적인 KV 세트")

        print("\n2️⃣ Grouped-Query Attention (GQA)")
        print("   Query: [H1] [H2] [H3] [H4] [H5] [H6] [H7] [H8]")
        print("          └─┬─┘ └─┬─┘ └─┬─┘ └─┬─┘")
        print("   Key:     [G1]   [G2]   [G3]   [G4]")
        print("   Value:   [G1]   [G2]   [G3]   [G4]")
        print("   → 4개의 그룹화된 KV 세트 (각 그룹에 2개 Query)")

        print("\n3️⃣ Multi-Query Attention (MQA)")
        print("   Query: [H1] [H2] [H3] [H4] [H5] [H6] [H7] [H8]")
        print("          └─────────┬─────────┘")
        print("   Key:           [SHARED]")
        print("   Value:         [SHARED]")
        print("   → 1개의 공유 KV 세트")

        print("\n" + "=" * 60)
        print("💡 핵심 트레이드오프")
        print("=" * 60)

        comparison = [
            ['특성', 'MHA', 'GQA', 'MQA'],
            ['KV 헤드 수', '많음 (H)', '중간 (G)', '적음 (1)'],
            ['메모리 사용량', '높음', '중간', '낮음'],
            ['표현력', '최고', '높음', '중간'],
            ['추론 속도', '느림', '중간', '빠름'],
            ['품질', '⭐⭐⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐'],
        ]

        for row in comparison:
            print(f"{row[0]:15s} | {row[1]:15s} | {row[2]:15s} | {row[3]:15s}")

        print("=" * 60)

# ========================================
# 사용 예시
# ========================================

gqa_explorer = GQAModelExplorer()

# Llama 2 70B 분석
gqa_explorer.load_llama2_gqa()

# Mistral 7B 분석
gqa_explorer.load_mistral_gqa()

# GQA 개념 시각화
gqa_explorer.visualize_gqa_concept()

### **Multi-Query Attention (MQA) 모델**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

class MQAModelExplorer:
    """
    Multi-Query Attention 모델 탐색 및 분석
    """
    def __init__(self):
        pass

    def load_falcon_mqa(self, model_size='7b'):
        """Falcon의 MQA 구조 분석"""
        if model_size == '40b':
            model_name = 'tiiuae/falcon-40b'
            num_heads = 64
            hidden_size = 8192
            num_layers = 60
            params = '40B'
        else:  # 7b
            model_name = 'tiiuae/falcon-7b'
            num_heads = 71
            hidden_size = 4544
            num_layers = 32
            params = '7B'

        print("\n" + "=" * 60)
        print(f"🔍 Falcon {model_size.upper()} (Multi-Query Attention) 분석")
        print("=" * 60)

        print(f"\n📊 모델: {model_name}")
        print(f"총 파라미터: {params}")

        print(f"\n🎯 MQA Attention 설정:")
        print(f"   Query 헤드 수: {num_heads}")
        print(f"   KV 헤드 수: 1 ⭐ (모든 Query가 공유!)")
        print(f"   Hidden size: {hidden_size}")
        print(f"   헤드당 차원: {hidden_size // num_heads}")

        print(f"\n🏗️  아키텍처:")
        print(f"   Decoder 레이어 수: {num_layers}")

        print(f"\n✨ Multi-Query Attention 특징:")
        print(f"   ✅ Query 헤드: {num_heads}개 (독립)")
        print(f"   ✅ Key 헤드: 1개 (전체 공유)")
        print(f"   ✅ Value 헤드: 1개 (전체 공유)")
        print(f"   → 극도로 효율적인 KV 캐시!")

        # 메모리 효율성 계산
        head_dim = hidden_size // num_heads
        seq_len = 2048

        # MHA vs MQA 메모리 비교
        mha_kv_cache = 2 * num_layers * num_heads * seq_len * head_dim * 2  # FP16
        mqa_kv_cache = 2 * num_layers * 1 * seq_len * head_dim * 2  # FP16

        print(f"\n💾 KV 캐시 메모리 비교 (시퀀스 길이 {seq_len}):")
        print(f"   MHA 방식: {mha_kv_cache / 1e9:.2f} GB")
        print(f"   MQA 방식: {mqa_kv_cache / 1e9:.2f} GB")
        print(f"   절감량: {(1 - mqa_kv_cache / mha_kv_cache) * 100:.1f}%")
        print(f"   → {num_heads}배 메모리 효율! 🚀")

        print(f"\n🚀 Falcon의 추가 최적화:")
        print(f"   ✅ FlashAttention")
        print(f"   ✅ Rotary Position Embeddings")
        print(f"   ✅ Parallel Attention/MLP")

        print("=" * 60)

    def compare_all_attention_mechanisms(self):
        """모든 Attention 메커니즘 종합 비교"""
        print("\n" + "=" * 60)
        print("📊 Attention 메커니즘 종합 비교")
        print("=" * 60)

        # 예시: 64개 Query 헤드, 32개 레이어, 4096 시퀀스
        num_query_heads = 64
        num_layers = 32
        seq_len = 4096
        head_dim = 128

        print(f"\n가정: Query 헤드 {num_query_heads}개, "
              f"레이어 {num_layers}개, 시퀀스 {seq_len}개")

        # MHA
        mha_kv_heads = num_query_heads
        mha_memory = 2 * num_layers * mha_kv_heads * seq_len * head_dim * 2 / 1e9

        # GQA (8개 그룹)
        gqa_kv_heads = 8
        gqa_memory = 2 * num_layers * gqa_kv_heads * seq_len * head_dim * 2 / 1e9

        # MQA
        mqa_kv_heads = 1
        mqa_memory = 2 * num_layers * mqa_kv_heads * seq_len * head_dim * 2 / 1e9

        print("\n" + "=" * 80)
        print(f"{'메커니즘':<15} {'KV 헤드':<10} {'메모리 (GB)':<15} {'절감률':<15} {'추론속도':<15}")
        print("=" * 80)
        print(f"{'MHA':<15} {mha_kv_heads:<10} {mha_memory:<15.2f} {'기준':<15} {'1.0x':<15}")
        print(f"{'GQA':<15} {gqa_kv_heads:<10} {gqa_memory:<15.2f} {f'{(1-gqa_memory/mha_memory)*100:.1f}%':<15} {'~2-3x':<15}")
        print(f"{'MQA':<15} {mqa_kv_heads:<10} {mqa_memory:<15.2f} {f'{(1-mqa_memory/mha_memory)*100:.1f}%':<15} {'~4-6x':<15}")
        print("=" * 80)

        print("\n" + "=" * 60)
        print("🎯 선택 가이드")
        print("=" * 60)

        recommendations = [
            {
                'use_case': '최고 품질 필요',
                'recommendation': 'MHA',
                'examples': 'BERT, GPT-3, T5'
            },
            {
                'use_case': '품질과 속도의 균형',
                'recommendation': 'GQA',
                'examples': 'Llama 2, Mistral, Mixtral'
            },
            {
                'use_case': '실시간 추론 중요',
                'recommendation': 'MQA',
                'examples': 'Falcon, PaLM, StarCoder'
            },
            {
                'use_case': '제한된 메모리 환경',
                'recommendation': 'MQA 또는 GQA',
                'examples': '모바일, 엣지 디바이스'
            },
            {
                'use_case': '대화형 AI (낮은 레이턴시)',
                'recommendation': 'MQA 또는 GQA',
                'examples': '챗봇, 실시간 번역'
            }
        ]

        for rec in recommendations:
            print(f"\n🎯 {rec['use_case']}")
            print(f"   권장: {rec['recommendation']}")
            print(f"   예시: {rec['examples']}")

        print("\n" + "=" * 60)
        print("💡 최신 트렌드 (2023-2025)")
        print("=" * 60)
        print("✅ GQA가 새로운 표준으로 부상")
        print("   → Llama 2, Mistral, Gemini 등 주요 모델 채택")
        print("✅ MHA는 소형 모델에서만 사용")
        print("   → BERT, GPT-2 등 기존 모델")
        print("✅ MQA는 특수 목적 (속도 중시) 모델에 사용")
        print("   → Falcon, StarCoder")
        print("=" * 60)

    def benchmark_inference_speed(self):
        """추론 속도 비교 시뮬레이션"""
        print("\n" + "=" * 60)
        print("⚡ 추론 속도 시뮬레이션")
        print("=" * 60)

        # 가상의 벤치마크 데이터
        batch_sizes = [1, 4, 16, 32]

        # 상대적 속도 (MHA 대비)
        mha_speed = [1.0, 1.0, 1.0, 1.0]
        gqa_speed = [2.2, 2.1, 1.9, 1.8]
        mqa_speed = [4.5, 4.2, 3.8, 3.5]

        print("\n시퀀스 길이 2048 토큰, 생성 512 토큰 가정")
        print("\n" + "-" * 60)
        print(f"{'Batch Size':<15} {'MHA':<15} {'GQA':<15} {'MQA':<15}")
        print("-" * 60)

        for i, bs in enumerate(batch_sizes):
            print(f"{bs:<15} {mha_speed[i]:<15.1f}x {gqa_speed[i]:<15.1f}x {mqa_speed[i]:<15.1f}x")

        print("-" * 60)
        print("\n💡 인사이트:")
        print("   - Batch size가 작을수록 MQA의 장점이 더 큼")
        print("   - 실시간 서비스(batch=1)에서 MQA는 4-5배 빠름")
        print("   - GQA는 속도와 품질의 최적 균형점")
        print("=" * 60)

# ========================================
# 사용 예시
# ========================================

mqa_explorer = MQAModelExplorer()

# Falcon 7B 분석
mqa_explorer.load_falcon_mqa('7b')

# Falcon 40B 분석
mqa_explorer.load_falcon_mqa('40b')

# 모든 메커니즘 비교
mqa_explorer.compare_all_attention_mechanisms()

# 추론 속도 벤치마크
mqa_explorer.benchmark_inference_speed()